### Importando bibliotecas

In [1]:
import pandas as pd
from sqlalchemy import create_engine, VARCHAR, Integer
import mysql.connector
from geopy.geocoders import Nominatim
import requests

### Extraindo arquivo com a lista de ceps - Cidade Poços de Caldas

In [2]:
def cep():
    # ler o arquivo xlsx
    lista_cep = pd.read_excel('lista_cep.xlsx')
    
    # Extraio 13 caracteres da esquerda, a saida será CEP 37701-000
    lista_cep['primeiros_treze'] = lista_cep['ceps'].str[:13]
    
    # Extraio agora os 9 caracteres do CEP
    lista_cep['cep'] = lista_cep['primeiros_treze'].str[-9:]
    
    # As colunas 'ceps' e 'primeiros_teze' não são mais necessárias
    lista_cep = lista_cep.drop(columns = ['ceps', 'primeiros_treze'])
    
    # Retiro o hifen
    lista_cep = lista_cep['cep'].str.replace('-', '').replace(' ', '')
    
    lista_cep = list(lista_cep)
    
    return lista_cep

### Consumindo API Via CEP

In [3]:
def consome_api(ceps):
    
    dados_ceps = []
    
    for cep in ceps:
        # Define url
        url = f'https://viacep.com.br/ws/{cep}/json/'

        # Faz a requisição e formata em json
        req = requests.get(url)
        req = req.json()

        # Extrai CEP, Localidade, Logradouro, UF e DDD
        cep = req['cep']
        localidade = req['localidade']
        logradouro = req['logradouro']
        uf = req['uf']
        ddd = req['ddd']
        log_loc = logradouro + ', ' + localidade

        # DataFrame com as colunas
        dados_ceps.append({ 'CEP': cep,
                            'Localidade': localidade,
                            'Logradouro': logradouro,
                            'Endereco': log_loc,
                            'UF': uf,
                            'DDD': ddd})

    df = pd.DataFrame(dados_ceps)


    return df

### Extraindo Coordenadas

In [4]:
def geocoder():
    
    # Chamamos a funcao consome api para usarmos o dataframe retornado por ela
    lista_cep = cep()
    dados_consome_api = consome_api(lista_cep)
        
    coordenadas_df = []
    
    for i in dados_consome_api['Endereco']:
        
        # Inicializa o geopy
        geo = Nominatim(user_agent = 'My-Geo')
        
        # Cria o endereço e executa a consulta
        location = geo.geocode(i)
        
        if location:
            
            # Coordenadas
            lat = location.latitude
            lon = location.longitude

            coordenadas_df.append({'Latitude': lat,
                                   'Longitude': lon})
        else:
            print('Não foi possível encontrar o endereço: ', i)
    
    coordenadas_df = pd.DataFrame(coordenadas_df)
    
    dados_consome_api['Latitude'] = coordenadas_df['Latitude']
    dados_consome_api['Longitude'] = coordenadas_df['Longitude']
    
    return dados_consome_api

In [5]:
geocoder()

,CEP,Localidade,Logradouro,Endereco,UF,DDD,Latitude,Longitude
0,37701-000,Poços de Caldas,Rua Assis Figueiredo,"Rua Assis Figueiredo, Poços de Caldas",MG,35,-21.792491,-46.566936
1,37701-001,Poços de Caldas,Rua Rio Grande do Sul,"Rua Rio Grande do Sul, Poços de Caldas",MG,35,-21.779386,-46.563502
2,37701-002,Poços de Caldas,Praça Doutor Pedro Sanches,"Praça Doutor Pedro Sanches, Poços de Caldas",MG,35,-21.786401,-46.568061
3,37701-004,Poços de Caldas,Rua Minas Gerais,"Rua Minas Gerais, Poços de Caldas",MG,35,-21.786552,-46.567718
4,37701-005,Poços de Caldas,Rua Goiás,"Rua Goiás, Poços de Caldas",MG,35,-21.780983,-46.567924
5,37701-006,Poços de Caldas,Rua Mato Grosso,"Rua Mato Grosso, Poços de Caldas",MG,35,-21.782941,-46.569673
6,37720-000,Botelhos,,", Botelhos",MG,35,-21.650006,-46.394736
7,37720-000,Botelhos,,", Botelhos",MG,35,-21.650006,-46.394736
8,37720-000,Botelhos,,", Botelhos",MG,35,-21.650006,-46.394736


### Inserindo dados desta tabela no Banco de Dados MySQL

In [6]:
def input_db():
    
    dados = geocoder()
    
    try:
        # Conecta ao banco de dados
        engine = create_engine('mysql+mysqlconnector://root:@localhost/portfolio')
        
        # Define os dtypes
        dtypes = {'CEP':        VARCHAR(50),
                  'Localidade': VARCHAR(50),
                  'Logradouro': VARCHAR(50),
                  'Endereco':   VARCHAR(150),
                  'UF':         VARCHAR(2),
                  'DDD':        Integer,
                  'Latitude':   VARCHAR(100),
                  'Longitude':  VARCHAR(100)}
        
        # insere os dados na tabela 'geo_location'
        dados.to_sql('geo_location', con = engine, if_exists = 'replace', index = False, dtype = dtypes)
        
        print('Dados inseridos com sucesso!')
        print('Total de registros inclusos: ', len(dados))
        
        # Consulta que retorna quantidade de registros na tabela
       # conn = engine.connect()
        query = 'SELECT count(1) FROM geo_location'
        result_query = engine.execute(query).fetchone()
        
        print('Total de registros na tabela: ', result_query[0])


    except mysql.connector.Error as err:
        print('Erro ao conectar ou interagir com o banco de dados.', err)

In [7]:
input_db()

Dados inseridos com sucesso!
Total de registros inclusos:  9
Total de registros na tabela:  9
